<a href="https://colab.research.google.com/github/battleship0000/NLP---online-shopping-chat-bot/blob/main/nlp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk spacy scikit-learn pandas --quiet

import pandas as pd
import re, string, nltk, spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


nlp = spacy.load("en_core_web_sm")



data = {
    "Question": [
        "What is the admission fee?",
        "How can I apply for bus?",
        "When will exams start?",
        "How can I check my attendance?",
        "Where can I find the timetable?",
        "What are the library timings?",
        "How can I get my ID card?",
        "Where is the admission office located?",
        "What are the placement opportunities?",
        "How to contact the student welfare department?",
        "When does the new semester begin?",
        "What is the last date to submit assignments?",
        "How can I pay my university fees?",
        "Is there any scholarship program?",
        "How can I join university clubs?",
        "What is the dress code in campus?",
        "Where can I collect my admit card?",
        "Who is the Dean of Student Welfare?",
        "How to get a transcript or bonafide certificate?",
        "What are the cafeteria timings?",
        "How do I access the online portal?",
        "How to reset my student portal password?",
        "Can I change my department or course?",
        "Where can I see the academic calendar?",
        "How can I get hostel Wi-Fi password?",
        "What should I do if I lose my ID card?",
        "Where can I check my internal marks?",
        "How can I contact the examination department?",
        "How to apply for a re-evaluation?",
        "How can I register for convocation?",
        "How can international students apply?",
        "What is the procedure for transfer certificate?",
        "How can I access the university transport facility?",
        "Who is the faculty advisor?",
        "How can I get in touch with the placement cell?"
    ],

    "Answer": [
        "The admission fee is ₹5000 for all undergraduate programs.",
        "You can apply for hostel through the online portal at hostel.university.edu.",
        "Exams will begin in December as per the academic calendar.",
        "You can check your attendance on the student portal under 'Academics'.",
        "The timetable is available on the student portal and notice board.",
        "The library is open from 9 AM to 6 PM on weekdays.",
        "You can get your ID card from the administrative office after verification.",
        "The admission office is located in Block A, Ground Floor.",
        "The university offers placement opportunities with over 200 reputed companies every year.",
        "You can contact the Student Welfare Department via email at welfare@university.edu.",
        "The new semester begins in the first week of August.",
        "Assignments should be submitted by the 20th of each month unless extended.",
        "Fees can be paid online via the ERP portal using UPI, NetBanking, or Credit Card.",
        "Yes, scholarships are available based on merit and financial need. Check scholarship.university.edu.",
        "You can join clubs by registering at studentactivities.university.edu.",
        "Students are expected to wear decent and formal attire inside the campus.",
        "Admit cards can be collected from the examination cell two weeks before exams.",
        "Dr. Anshul Saluja is the Dean of Student Welfare.",
        "You can apply for a transcript or bonafide certificate from the admin section on the ERP portal.",
        "The cafeteria is open from 8 AM to 8 PM.",
        "Log in to the university portal at portal.university.edu using your student ID.",
        "You can reset your password using the 'Forgot Password' option on the portal.",
        "Course or department changes can be requested before the semester starts through the academic office.",
        "The academic calendar is uploaded every semester on the official website.",
        "You can get the hostel Wi-Fi password from the hostel warden.",
        "Report the loss immediately to the admin office to get a duplicate ID card.",
        "Internal marks are available on the ERP portal under 'Results' section.",
        "You can contact the exam department at examcell@university.edu.",
        "Apply for re-evaluation via ERP portal under 'Exam Services'.",
        "Convocation registration opens a month before the ceremony on university.edu/convocation.",
        "International students can apply through the global admissions page at global.university.edu.",
        "Apply for a transfer certificate at the admin office or through ERP > Certificates.",
        "Transport routes and passes are managed via transport.university.edu.",
        "The faculty advisor is assigned by your department at the start of the semester.",
        "You can reach the placement cell via placement@university.edu or visit Block C, Room 210."
    ]
}

faq_data = pd.DataFrame(data)



lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = nltk.word_tokenize(text)
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(clean_tokens)

faq_data["cleaned_question"] = faq_data["Question"].apply(preprocess_text)



vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(faq_data["cleaned_question"])



def chatbot_response(user_query):
    user_clean = preprocess_text(user_query)
    user_tfidf = vectorizer.transform([user_clean])
    similarity_scores = cosine_similarity(user_tfidf, tfidf_matrix)

    best_match_index = similarity_scores.argmax()
    best_score = similarity_scores[0][best_match_index]

    if best_score < 0.3:
        return "I'm sorry, I didn't quite get that. Please contact the help desk at help@university.edu."

    return faq_data.iloc[best_match_index]["Answer"]



print("🎓 Welcome to the University FAQ Chatbot!")
user_name = input("Please enter your name: ")
print(f"Hello {user_name}, how can I assist you today? (Type 'exit' to quit)\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print(f"Chatbot: Goodbye, {user_name}! Have a great day! 🌟")
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)
